## Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from kmodes.kmodes import KModes

## Limpeza e Pré processamento

### Carregamento

In [ ]:
desired_cols = [

  "SG_UF",

  "ANO_NASC",
  "CS_SEXO",
  "CS_RACA",
  "CS_ESCOL_N",
  "CS_GESTANT",

  "FEBRE",    # Sintomas
  "MIALGIA",
  "CEFALEIA",
  "EXANTEMA",
  "VOMITO",
  "NAUSEA",
  "DOR_COSTAS",
  "CONJUNTVIT",
  "ARTRITE",
  "ARTRALGIA",
  "PETEQUIA_N",
  "LEUCOPENIA",
  "LACO",
  "DOR_RETRO",

  "DIABETES",   # Comorbidades
  "HEMATOLOG",
  "HEPATOPAT",
  "RENAL",
  "HIPERTENSA",
  "ACIDO_PEPT",
  "AUTO_IMUNE",

  "HOSPITALIZ", # Desfecho
  "CLASSI_FIN", 
  "EVOLUCAO",
]

# Carregando dataset
DATA_PATH = "./data/original-2/DENGBR24.csv"


chunks = []

for chunk in pd.read_csv(DATA_PATH, low_memory=False, usecols=desired_cols, chunksize=500_000):
  filtered = chunk[
    (chunk["CLASSI_FIN"].isin([10, 11, 12]))  &
    (chunk["SG_UF"].isin([31, 32, 33, 35]))   &

    (chunk["ANO_NASC"].notna())               &
    (chunk["ANO_NASC"] > 1924)                &
    
    (chunk["CS_SEXO"].isin(["M","F"]))        &   
    (chunk["CS_RACA"].isin([1, 2, 3, 4, 5]))  &  
    (chunk["CS_ESCOL_N"].isin([0, 1, 2, 3, 4, 5, 6, 7, 8]))              
  ]
  
  chunks.append(filtered)


df = pd.concat(chunks, ignore_index=True)
df.head()

### Tratamento

In [ ]:
df.shape

In [ ]:
df["CS_GESTANT"] = df["CS_GESTANT"].replace({
  5: 0,
  6: 0,
  9: 0,
  1: 1,
  2: 1,
  3: 1,
  4: 1
})
df["CS_GESTANT"] = df["CS_GESTANT"].fillna(0)
df["CS_GESTANT"] = df["CS_GESTANT"].astype(int)

In [ ]:
current_year =2024
df["IDADE"] = current_year - df["ANO_NASC"]
df["FAIXA_IDADE"] = pd.cut(df["IDADE"], bins=[0, 10, 20, 30, 40, 60, 100], labels=["0-10", "11-20", "21-30", "31-40", "41-60", "60+"])

In [ ]:
df_dummies = pd.get_dummies(df, columns=["FAIXA_IDADE", "CS_SEXO", "CS_RACA", "CS_ESCOL_N"])
df_dummies.info()

## Clusterização

In [ ]:
clustering_cols = [
  "CS_RACA_1.0",
  "CS_RACA_2.0",
  "CS_RACA_3.0",
  "CS_RACA_4.0",
  "CS_RACA_5.0",

  "CS_SEXO_M",

  "CS_ESCOL_N_0.0",
  "CS_ESCOL_N_1.0",
  "CS_ESCOL_N_2.0",
  "CS_ESCOL_N_3.0",
  "CS_ESCOL_N_4.0",
  "CS_ESCOL_N_5.0",
  "CS_ESCOL_N_6.0",
  "CS_ESCOL_N_7.0",
  "CS_ESCOL_N_8.0",

  "FAIXA_IDADE_0-10",
  "FAIXA_IDADE_11-20",
  "FAIXA_IDADE_21-30",
  "FAIXA_IDADE_31-40",
  "FAIXA_IDADE_41-60",
  "FAIXA_IDADE_60+",

  "CS_GESTANT"
]

In [ ]:
df_clustering = df_dummies[clustering_cols]
df_clustering.head()

In [ ]:
sample = df_clustering.sample(frac=0.2)

costs = []
K_range = range(1, 11)

for k in K_range:
    km = KModes(n_clusters=k, init='Huang', n_init=1, verbose=1, random_state=10)
    km.fit_predict(sample)
    costs.append(km.cost_)

plt.plot(K_range, costs, marker='o')
plt.xlabel('Número de clusters (K)')
plt.ylabel('Custo (Distância intra-cluster)')
plt.title('Método do Cotovelo para K-Modes')
plt.grid()
plt.show()

In [ ]:
# Crie o modelo com K=5 e uma inicialização robusta
km = KModes(n_clusters=3, init='Huang', n_init=3, verbose=1, random_state=1)

# Ajuste o modelo aos dados completos e obtenha os clusters
clusters = km.fit_predict(df_clustering)  # df = seu dataframe categórico

# Agora você pode adicionar os clusters no dataframe
df_clustering['cluster'] = clusters

In [ ]:
from sklearn.decomposition import TruncatedSVD

X = df_clustering.values  # pode ser matriz esparsa, ex: scipy.sparse.csr_matrix

svd = TruncatedSVD(n_components=2, random_state=1)
X_svd = svd.fit_transform(X)

plt.figure(figsize=(10, 7))
scatter = plt.scatter(X_svd[:, 0], X_svd[:, 1], c=df_clustering['cluster'], cmap='tab10', s=10)
plt.legend(*scatter.legend_elements(), title="Clusters")
plt.title("Visualização dos clusters com TruncatedSVD")
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.show()

In [ ]:
# TruncatedSVD para 3 componentes
svd = TruncatedSVD(n_components=3, random_state=1)
X_svd_3d = svd.fit_transform(X)

# Plot 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(
    X_svd_3d[:, 0], X_svd_3d[:, 1], X_svd_3d[:, 2],
    c=df_clustering['cluster'], cmap='tab10', s=15, alpha=0.8
)

ax.set_title("Visualização 3D dos clusters com TruncatedSVD")
ax.set_xlabel("Componente 1")
ax.set_ylabel("Componente 2")
ax.set_zlabel("Componente 3")

legend1 = ax.legend(*scatter.legend_elements(), title="Clusters")
ax.add_artist(legend1)

plt.show()

In [ ]:
df["cluster"] = clusters
df["cluster"].value_counts()

In [ ]:
df.groupby('cluster')['IDADE'].mean().plot(kind='bar')
plt.title('Média de Idade por Cluster')
plt.ylabel('Idade média')
plt.show()

In [ ]:
import seaborn as sns

cluster_profile = df_clustering.groupby('cluster').mean()

plt.figure(figsize=(10, 20))
sns.heatmap(cluster_profile.T, cmap='rocket_r', annot=False, fmt=".2f")
plt.title('Proporção média das categorias por cluster')
plt.xlabel('Categorias One-Hot')
plt.ylabel('Cluster')
plt.show()

In [ ]:
hospitaliz_table = pd.crosstab(df["cluster"], df["HOSPITALIZ"])
hospitaliz_table.columns = ["Sim", "Não", "Ignorado"]
hospitaliz_prop = hospitaliz_table.div(hospitaliz_table.sum(axis=1), axis=0)
hospitaliz_prop = hospitaliz_prop * 100

plt.figure(figsize=(8, 6))
sns.heatmap(hospitaliz_prop, annot=hospitaliz_prop.round(2).astype(str) + '%', cmap="OrRd", fmt="")
plt.title("Proporção de Tipos de Hospitalização por Cluster")
plt.xlabel("Tipo de Hospitalização")
plt.ylabel("Cluster")
plt.show()

In [ ]:
evolution_table = pd.crosstab(df["cluster"], df["EVOLUCAO"])
# evolution_table.columns = ["cura", "óbito pelo agravo", "óbito por outras causas", "ignorado"]
evolution_prop = evolution_table.div(evolution_table.sum(axis=1), axis=0)
evolution_prop = evolution_prop * 100

plt.figure(figsize=(8, 5))
sns.heatmap(evolution_prop, annot=evolution_prop.round(2).astype(str) + '%', cmap="OrRd", fmt="")
plt.title("Proporção de Tipos de Evolução por Cluster")
plt.xlabel("Tipo de Evolução")
plt.ylabel("Cluster")
plt.show()

In [ ]:
classi_table = pd.crosstab(df["cluster"], df["CLASSI_FIN"])
# classi_table.columns = ["inconclusivo", "dengue", "dengue alarmante", "dengue grave"]
classi_prop = classi_table.div(classi_table.sum(axis=1), axis=0)
classi_prop = classi_prop * 100

plt.figure(figsize=(8, 5))
sns.heatmap(classi_prop, annot=classi_prop.round(2).astype(str) + '%', cmap="rocket_r", fmt="")
plt.title("Proporção de Tipos de Dengue por Cluster")
plt.xlabel("Tipo de Evolução")
plt.ylabel("Cluster")
plt.show()

In [ ]:
raca_table = pd.crosstab(df["cluster"], df["CS_RACA"])
raca_prop = raca_table.div(raca_table.sum(axis=1), axis=0)
raca_prop.columns = ["branca", "preta", "amarela", "parda", "indigena"]
raca_prop = raca_prop * 100

plt.figure(figsize=(8, 5))
sns.heatmap(raca_prop, annot=raca_prop.round(2).astype(str) + '%', cmap="OrRd", fmt="")
plt.title("Proporção de Raças por Cluster")
plt.xlabel("Raças")
plt.ylabel("Cluster")
plt.show()

In [ ]:
escol_table = pd.crosstab(df["cluster"], df["CS_ESCOL_N"])
escol_prop = escol_table.div(escol_table.sum(axis=1), axis=0)
# escol_prop.columns = ["branca", "preta", "amarela", "parda", "indigena"]
escol_prop = escol_prop * 100

plt.figure(figsize=(15, 5))
sns.heatmap(escol_prop, annot=escol_prop.round(2).astype(str) + '%', cmap="OrRd", fmt="")
plt.title("Proporção de Escolaridade por Cluster")
plt.xlabel("Nível de Escolaridade")
plt.ylabel("Cluster")
plt.show()